In [1]:
import numpy as np
import pandas as pd
import os
import re
from textblob import TextBlob
from sklearn import preprocessing
from catboost import CatBoostClassifier, CatBoostRegressor, CatBoost, Pool
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score, accuracy_score, mean_absolute_error
from sklearn.model_selection import train_test_split


ModuleNotFoundError: No module named 'textblob'

In [ ]:
def clear_text(text):
    text = re.sub('[^a-zA-Z]', ' ', text)
    text = text.lower()
    return text


In [66]:
def label_encoding(train: pd.DataFrame, test: pd.DataFrame, col_definition: dict):
    """
    col_definition: encode_col
    """
    n_train = len(train)
    train = pd.concat([train, test], sort=False).reset_index(drop=True)
    for f in col_definition['encode_col']:
        try:
            lbl = preprocessing.LabelEncoder()
            train[f] = lbl.fit_transform(list(train[f].values))
        except:
            print(f)
    test = train[n_train:].reset_index(drop=True)
    train = train[:n_train]
    return train, test

In [67]:
#urls
def change_url(label):
  m = re.search('https?://([A-Za-z_0-9.-]+)/', label)
  if m:
    label = m.group(1)
  return(label)  

In [68]:
#transtate
def traslate_text(text):
 try:
   text_ret = TextBlob(text).translate(to='en')
 except Exception as e:
   text_ret = text
 return(str(text_ret))

In [69]:
train = pd.read_csv('E:/juniper_notebooks/Task1_NLP/input/articles_train.csv')
test = pd.read_csv('E:/juniper_notebooks/Task1_NLP/input/articles_test.csv')
sub = pd.read_csv('E:/juniper_notebooks/Task1_NLP/input/sample_submission.csv')

In [70]:
train['claps'].fillna(0)
train['claps'] = train['claps'].astype(int)
train['claps'] = train['claps'].map(lambda b: (b // 50)*50)

train['reading_time'].fillna(0)
train['reading_time'] = train['reading_time'].astype(int)
train['reading_time'] = train['reading_time'].map(lambda b: (b // 5)*5)

train['link'] = train['link'].map(lambda b: change_url(b))
test['link'] = test['link'].map(lambda b: change_url(b))

train['text'] = train['text'].map(lambda b: traslate_text(b))
test['text'] = test['text'].map(lambda b: traslate_text(b))

train['text'] = train['text'].map(lambda b: clear_text(b))
test['text'] = test['text'].map(lambda b: clear_text(b))

In [71]:
def value_to_float(x):
    if type(x) == float or type(x) == int:
        return x
    if 'K' in x:
        if len(x) > 1:
            return float(x.replace('K', '')) * 1000
        return 1000.0
    if 'M' in x:
        if len(x) > 1:
            return float(x.replace('M', '')) * 1000000
        return 1000000.0
    if 'B' in x:
        return float(x.replace('B', '')) * 1000000000
    return 0.0

def findFollwers(text):
 m = re.search('([A-Za-z_0-9.-]+) Followers', text)
 if m:
    sum = m[1]
    return sum
 
 return "0"  

train['folowers'] = train['text'].apply(findFollwers).apply(value_to_float)
test['folowers'] = train['text'].apply(findFollwers).apply(value_to_float)

In [72]:
target_col = 'claps'
text_cols = ['text']
categorical_cols = ['author', 'link']
num_cols = ['reading_time', 'folowers']

In [73]:
train, test = label_encoding(train, test, col_definition={'encode_col': categorical_cols})

In [74]:
X_train = train[text_cols + categorical_cols + num_cols]
y_train = train[target_col].values
X_test = test[text_cols + categorical_cols + num_cols]

In [100]:
#OOF
y_preds = []
models = []
oof_train = np.zeros((len(X_train),))
cv = StratifiedKFold(n_splits=30, shuffle=True, random_state=0)

catboost_params = {
    'task_type': 'GPU',
    'early_stopping_rounds': 10,
    'use_best_model': True,
    'verbose': False,
    'class_names': y_train
}

for fold_id, (train_index, valid_index) in enumerate(cv.split(X_train, y_train)):
    
    X_tr = X_train.loc[train_index, :]
    X_val = X_train.loc[valid_index, :]
    y_tr = y_train[train_index]
    y_val = y_train[valid_index]
    
    train_pool = Pool(
        X_tr, 
        y_tr, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=list(X_tr)
    )
    valid_pool = Pool(
        X_val, 
        y_val, 
        cat_features=categorical_cols,
        text_features=text_cols,
        feature_names=list(X_tr)
    )

    model = CatBoostClassifier(**catboost_params)
    model.fit(train_pool, eval_set=valid_pool)
    y_preds.append(model.predict(X_test))
                   
    models.append(model)

E:\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:666: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=30.
  warnings.warn(("The least populated class in y has only %d"
Custom logger is already specified. Specify more than one logger at same time is not thread safe.Found only 138 unique classes in the data, but have defined 3756 classes. Probably something is wrong with data.
Label(s) 3087, 1443 are not present in the train set. Perhaps, something is wrong with the data.
Found only 139 unique classes in the data, but have defined 3756 classes. Probably something is wrong with data.
Label(s) 3162 are not present in the train set. Perhaps, something is wrong with the data.
Found only 139 unique classes in the data, but have defined 3756 classes. Probably something is wrong with data.
Label(s) 3364 are not present in the train set. Perhaps, something is wrong with the data.
Found only 138 unique classes in the data, but have defin

In [101]:
y_sub = sum(y_preds) / len(y_preds)

In [138]:
sub

,claps
id,
3756,58
3757,8600
3758,1
3759,0
3760,0
...,...
4251,0
4252,110
4253,48


In [135]:
sub['claps'] = np.reshape(y_sub, (500, 1)).astype('int')

In [129]:
sub.to_csv('submission.csv')